In [1]:
import numpy as np
import pandas as pd

In [2]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('C:\data\RS\u.data.csv', sep='\t', names=header)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100003 entries, 0 to 100002
Data columns (total 4 columns):
user_id      100003 non-null int64
item_id      100003 non-null int64
rating       100003 non-null int64
timestamp    100003 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [4]:
df.head(25)

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
5,22,377,1,878887116
6,244,51,2,880606923
7,166,346,1,886397596
8,298,474,4,884182806
9,115,265,2,881171488


In [5]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)

Number of users = 944 | Number of movies = 1682


In [6]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
from sklearn import cross_validation as cv
train_data_50, test_data_50 = cv.train_test_split(df, test_size=0.50)

In [8]:
from sklearn import cross_validation as cv
train_data_75, test_data_75 = cv.train_test_split(df, test_size=0.75)

In [9]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [10]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix_50 = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix_50 [line[1]-1, line[2]-1] = line[3]

test_data_matrix_50 = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix_50[line[1]-1, line[2]-1] = line[3]

In [11]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix_75 = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix_75[line[1]-1, line[2]-1] = line[3]

test_data_matrix_75 = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix_75[line[1]-1, line[2]-1] = line[3]

In [12]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [13]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity_50 = pairwise_distances(train_data_matrix_50, metric='cosine')
item_similarity_50 = pairwise_distances(train_data_matrix_50.T, metric='cosine')

In [14]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity_75 = pairwise_distances(train_data_matrix_75, metric='cosine')
item_similarity_75 = pairwise_distances(train_data_matrix_75.T, metric='cosine')

In [15]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [16]:
item_prediction_25 = predict(train_data_matrix, item_similarity, type='item')
user_prediction_25 = predict(train_data_matrix, user_similarity, type='user')
item_prediction_50 = predict(train_data_matrix_50, item_similarity_50, type='item')
user_prediction_50 = predict(train_data_matrix_50, user_similarity_50, type='user')
item_prediction_75 = predict(train_data_matrix_75, item_similarity_75, type='item')
user_prediction_75 = predict(train_data_matrix_75, user_similarity_75, type='user')


In [17]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [18]:
print 'User-based CF MSE=25%: ' + str(rmse(user_prediction_25, test_data_matrix))
print 'Item-based CF RMSE=25%: ' + str(rmse(item_prediction_25, test_data_matrix))
print("\n")

print 'User-based CF MSE=50%: ' + str(rmse(user_prediction_50, test_data_matrix_50))
print 'Item-based CF RMSE=50%: ' + str(rmse(item_prediction_50, test_data_matrix_50))
print("\n")

print 'User-based CF MSE=75%: ' + str(rmse(user_prediction_75, test_data_matrix_75))
print 'Item-based CF RMSE=75%: ' + str(rmse(item_prediction_75, test_data_matrix_75))

User-based CF MSE=25%: 3.12631841228
Item-based CF RMSE=25%: 3.45403142889


User-based CF MSE=50%: 3.12631841228
Item-based CF RMSE=50%: 3.45403142889


User-based CF MSE=75%: 3.12631841228
Item-based CF RMSE=75%: 3.45403142889
